## Two Site 1D Hubbard QITE
This notebook is a guide to implement QITE on the Fermi-Hubbard model with two sites

We will refer to Pauli matrices by their indices: $[I, X, Y, Z] \equiv [0, 1, 2, 3]$
For consistency in notation, we use the following mapping for 2-qubit operators, composed of a Pauli matrices acting on each qubit. This is the indexing used throughout the code.

In [1]:
pauli_indices = [0, 1, 2, 3]
pauli_pairs = [(i, j) for i in pauli_indices for j in pauli_indices]
print(pauli_pairs)
pauli_pair_dict = {pauli_pairs[i] : i for i in range(len(pauli_pairs))}
print(pauli_pair_dict)

[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)]
{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (1, 0): 4, (1, 1): 5, (1, 2): 6, (1, 3): 7, (2, 0): 8, (2, 1): 9, (2, 2): 10, (2, 3): 11, (3, 0): 12, (3, 1): 13, (3, 2): 14, (3, 3): 15}


We define a measurement function **measure** and a propagation function **propagate**

In **measure** we define circuits to measure the expectation value of any Pauli string

In **propagate**, we loop through the different values store in alist to construct the states. alist is stored as a list of lists and the indices are $a[timestep][gate]$. For a timestep $\Delta \tau$, the gate indices of $a$ correspond to:

$$
a[0]=e^{-ia[II]\Delta\tau\hat{I} \otimes \hat{I}} \text{,  } \; 
a[1]=e^{-ia[IX]\Delta\tau\hat{I} \otimes \hat{X}} \text{,  } \;
a[2]=e^{-ia[IY]\Delta\tau\hat{I} \otimes \hat{Y}} \text{,  } \;
a[3]=e^{-ia[ZZ]\Delta\tau\hat{I} \otimes \hat{Z}}
$$
$$
a[4]=e^{-ia[XI]\Delta\tau\hat{X} \otimes \hat{I}} \text{,  } \; 
a[5]=e^{-ia[XX]\Delta\tau\hat{X} \otimes \hat{X}} \text{,  } \;
a[6]=e^{-ia[XY]\Delta\tau\hat{X} \otimes \hat{Y}} \text{,  } \;
a[7]=e^{-ia[XZ]\Delta\tau\hat{X} \otimes \hat{Z}}
$$
$$
a[8]=e^{-ia[YI]\Delta\tau\hat{Y} \otimes \hat{I}} \text{,  } \; 
a[9]=e^{-ia[YX]\Delta\tau\hat{Y} \otimes \hat{X}} \text{,  } \;
a[10]=e^{-ia[YY]\Delta\tau\hat{Y} \otimes \hat{Y}} \text{,  } \;
a[11]=e^{-ia[YZ]\Delta\tau\hat{Y} \otimes \hat{Z}}
$$
$$
a[12]=e^{-ia[ZI]\Delta\tau\hat{Z} \otimes \hat{I}} \text{,  } \; 
a[13]=e^{-ia[ZX]\Delta\tau\hat{Z} \otimes \hat{X}} \text{,  } \;
a[14]=e^{-ia[ZY]\Delta\tau\hat{Z} \otimes \hat{Y}} \text{,  } \;
a[15]=e^{-ia[ZZ]\Delta\tau\hat{Z} \otimes \hat{Z}}
$$

The 0 index stores the coefficient for the identity matrix $\hat{I}$ on both qubits, which is a global phase for each qubit that we can ignore.

In [2]:
from pyquil.gates import *
from pyquil.noise import estimate_bitstring_probs,correct_bitstring_probs
from pyquil import Program,get_qc

def measure(p, ro, idx, qc, qbits):
    # Circuit to measure the expectation value of any Pauli string
    # For 2-qubit Pauli measurements, see https://docs.microsoft.com/en-us/quantum/concepts/pauli-measurements
    # initialize qc    
    qc = get_qc(qc)
    # measure II
    if idx == 0:
        return 1
    # measure IX
    elif idx == 1:
        p += SWAP(qbits[0], qbits[1])
        p += H(qbits[0])
    # measure IY
    elif idx == 2:
        p += SWAP(qbits[0], qbits[1])
        p += RX(np.pi/2, qbits[0])
    # measure IZ
    elif idx == 3:
        p += SWAP(qbits[0], qbits[1])

    # measure XI
    elif idx == 4:
        p += H(qbits[0])
    # measure XX
    elif idx == 5:
        p += H(qbits[0])
        p += H(qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure XY
    elif idx == 6:
        p += H(qbits[0])
        p += RX(np.pi/2, qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure XZ
    elif idx == 7:
        p += H(qbits[0])
        p += CNOT(qbits[1], qbits[0])
    # measure YI
    elif idx == 8:
        p += RX(np.pi/2, qbits[0])
    # measure YX
    elif idx == 9:
        p += RX(np.pi/2, qbits[0])
        p += H(qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure YY
    elif idx == 10:
        p += RX(np.pi/2, qbits[0])
        p += RX(np.pi/2, qbits[1])
        p += CNOT(qbits[1], qbits[0])
    #measure YZ
    elif idx == 11:
        p += RX(np.pi/2, qbits[0])
        p += CNOT(qbits[1], qbits[0])
    # measure ZI
    elif idx == 12:
        return 1
    # measure ZX
    elif idx == 13:
        p += H(qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure ZY
    elif idx == 14:
        p += RX(np.pi/2, qbits[1])
        p += CNOT(qbits[1], qbits[0])
    # measure ZZ
    elif idx == 15:
        p += CNOT(qbits[1], qbits[0])
    # oops
    else:
        raise ValueError

    p += MEASURE(qbits[0], ro[0])
    exe = qc.compile(p)
    res = qc.run(exe)
    probs = estimate_bitstring_probs(res)
    # probs = correct_bitstring_probs(probs,[correction_matrix])
    return probs[0] - probs[1]


In [3]:
def applyPhase(p,qbits,angle):
    p += CNOT(qbits[0],qbits[1])
    p += RZ(angle,qbits[1])
    p += CNOT(qbits[0],qbits[1])

In [4]:
def propagate(p,alist,qbits):
    # Circuit to propagate the state
    if len(alist) == 0:
        None
    else:
        for t in range(len(alist)):
            
            for i in range(1,16):
                angle = np.real(alist[t][i])
                # II... we skip this case because it's just a phase
                if i == 0:
                    continue
                # IX
                elif i == 1:
                    p += RX(angle,qbits[1])

                # IY
                elif i == 2:
                    p += RY(angle,qbits[1])

                # IZ
                elif i == 3:
                    p += RZ(angle,qbits[1])

                # XI
                elif i == 4:
                    p += RX(angle,qbits[0])
                    
                # XX
                elif i == 5:
                    p += H(qbits[0])
                    p += H(qbits[1])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[1])
                    p += H(qbits[0])
                    
                # XY
                elif i == 6:
                    p += H(qbits[0])
                    p += RX(np.pi/2, qbits[1])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[1])
                    p += H(qbits[0])
                    
                # XZ
                elif i == 7:
                    p += H(qbits[0])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[0])
                    
                # YI
                elif i == 8:
                    p += RY(angle,qbits[0])
                    
                # YX
                elif i == 9:
                    p += RX(np.pi/2, qbits[0])
                    p += H(qbits[1])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[1])
                    p += RX(-np.pi/2, qbits[0])
                    
                # YY
                elif i == 10:
                    p += RX(np.pi/2, qbits[0])
                    p += RX(np.pi/2, qbits[1])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[1])
                    p += RX(-np.pi/2, qbits[0])
                    
                # YZ
                elif i == 11:
                    p += RX(np.pi/2, qbits[0])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[0])
                    
                # ZI
                elif i == 12:
                    p += RZ(angle,qbits[0])
                    
                # ZX
                elif i == 13:
                    p += H(qbits[1])
                    applyPhase(p,qbits,angle)
                    p += H(qbits[1])
                    
                # ZY
                elif i == 14:
                    p += RX(np.pi/2, qbits[1])
                    applyPhase(p,qbits,angle)
                    p += RX(-np.pi/2, qbits[1])
                    
                # ZZ
                elif i == 15:
                    applyPhase(p,qbits,angle)
                    
                # oops
                else:
                    raise ValueError
                

We now want to obtain the coefficients a[m] at the current time step and append to alist. For the 1-qubit case, we need to construct the matrix $S_{ij} = \langle \psi | \sigma_{i} \sigma_{j} | \psi \rangle$ and the vector $b_{i}$. Well, our earlier functions allow us to measure the expectation values of the different pauli matrices. How do we obtain $S_{ij}$ from a list of $\langle \psi | \sigma | \psi \rangle$? We can exploit the fact that up to some coefficients, $\sigma_{i}\sigma_{j} = c_{ij}\sigma_{ij}$. For example, $\sigma_{x}\sigma_{y} = i\sigma_{z}$. We will need a matrix to keep track of what pauli matrix and coefficient we get for $\sigma_{i}\sigma_{j}$. These are stored in the matrices **idx** and **coeff** below.


For the 2-qubit case, we can reuse these matrices. For example,
$(X \otimes X) (X \otimes Y) = XX \otimes XY = I \otimes i Z$

In [5]:
import sys
sys.path.append('classical')

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm 
from binary_functions import Int2Bas,Bas2Int,Opp2Str,Str2Opp
from Pmn import PPmunu

In [7]:
# First populate the Lie algebra rules
index = np.zeros([4**2,4**2],dtype=int)
coeff = np.zeros([4**2,4**2],dtype=complex)

row = 0
for i in range(4**2):
    column = 0
    for j in range(4**2):
        Pnu = Opp2Str(Int2Bas(column,4,2))
        Pmu = Opp2Str(Int2Bas(row,4,2))
        A = Pmu[0] + Pnu[0]
        B = Pmu[1] + Pnu[1]
        A,intA = PPmunu(A)
        B,intB = PPmunu(B)
        index[i,j] = Bas2Int(Str2Opp(A+B),4)
        coeff[i,j] = intA*intB
        column += 1
    row += 1
    
print(index)
print(coeff)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
 [ 1  0  3  2  5  4  7  6  9  8 11 10 13 12 15 14]
 [ 2  3  0  1  6  7  4  5 10 11  8  9 14 15 12 13]
 [ 3  2  1  0  7  6  5  4 11 10  9  8 15 14 13 12]
 [ 4  5  6  7  0  1  2  3 12 13 14 15  8  9 10 11]
 [ 5  4  7  6  1  0  3  2 13 12 15 14  9  8 11 10]
 [ 6  7  4  5  2  3  0  1 14 15 12 13 10 11  8  9]
 [ 7  6  5  4  3  2  1  0 15 14 13 12 11 10  9  8]
 [ 8  9 10 11 12 13 14 15  0  1  2  3  4  5  6  7]
 [ 9  8 11 10 13 12 15 14  1  0  3  2  5  4  7  6]
 [10 11  8  9 14 15 12 13  2  3  0  1  6  7  4  5]
 [11 10  9  8 15 14 13 12  3  2  1  0  7  6  5  4]
 [12 13 14 15  8  9 10 11  4  5  6  7  0  1  2  3]
 [13 12 15 14  9  8 11 10  5  4  7  6  1  0  3  2]
 [14 15 12 13 10 11  8  9  6  7  4  5  2  3  0  1]
 [15 14 13 12 11 10  9  8  7  6  5  4  3  2  1  0]]
[[ 1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j
   1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j  1.+0.j]
 [ 1.+0.j  1.+0.j  0.+1.j  0.-1.j  1.+0.j  1.+0.j 

We need to calculate:

$$S_{ij} = \langle \psi | Q_{i} Q_{j} | \psi \rangle = \langle \psi | ( \sigma_{i0} \otimes \sigma_{i1} ) ( \sigma_{j0} \otimes \sigma_{j1} ) | \psi \rangle = \langle \psi | \left( \sigma_{i0} \sigma_{j0} \otimes \sigma_{i1} \sigma_{j1} \right) | \psi \rangle$$

And the vector 
$$b_{i} = i \langle \psi | Q_{i} | \Delta_{0} \rangle - h.c. = i \langle \psi | {( \sigma_{i0} \otimes \sigma_{i1})} | \Delta_{0} \rangle - h.c.$$

where $| \Delta_{0} \rangle = \frac{(| \psi' \rangle - | \psi \rangle)}{\Delta \tau}$

Using this, we can construct the S matrix and b vector. We use the **idx** and **coeff** matrices and the function np.linalg.std to solve for x in **update_alist**. Because the rotation gate is defined as $e^{-i\theta/2\sigma}$, we multiply the x by 2 before storing them in alist.

In [8]:
# a matrix should be 16 x 16
def update_alist(sigma_expectation,alist,db,delta,hm):
    '''
    Obtain A[m]
    To do this, we compute the S matrix and the b vector. We also need to compute the norm c
    Details:
    Each local Hamiltonian term hm can be a sum of many 2-qubit operators...
     ... see the construction of the hm_list to see why... for each term in hm (hm[i]):
    hm[i][1][0] is the multiplicative constant in the Hamiltonian
    hm[i][0][0] is the index of the Pauli matrix for qubit 0
    hm[i][0][1] is the index of the Pauli matrix for qubit 1
    '''
    
    # c is the squared norm
    c = 1
    for i in range(len(hm)):
        hm_idx = pauli_pair_dict[(hm[i][0][0], hm[i][0][1])]
        c -= 2 * db * hm[i][1][0] * sigma_expectation[hm_idx]
    # c is now the norm, after we take its square root
    c = np.sqrt(c)
        
    # Initialize S matrix
    S = np.zeros([16, 16], dtype=complex)
    # Initialize b vector
    b = np.zeros([16], dtype=complex)
    
    for i in range(16):
        # Step 1: Obtain S matrix
        for j in range(16):
            S[i, j] = sigma_expectation[index[i,j]] * coeff[i,j]
            
        # Step 2: Obtain b vector
        b[i] += (sigma_expectation[i]/c-sigma_expectation[i])/(db)
        # iterate through hm terms
        for j in range(len(hm)):
            hm_idx = pauli_pair_dict[(hm[j][0][0],hm[j][0][1])]            
            b[i] -= hm[j][1][0]*coeff[i,hm_idx]*sigma_expectation[index[i,hm_idx]]/c
        b[i] = 1j*b[i] - 1j*np.conj(b[i])
        
    # Step 3: Add regularizer... 16 x 16 matrix with 1s on main diagonal and 0s elsewhere
    dalpha = np.eye(16)*delta

    # Step 4: Solve for linear equation, the solution is multiplied by -2 because of the definition of unitary rotation gates is exp(-i theta/2)
    x = np.linalg.lstsq(S + np.transpose(S) + dalpha, -b, rcond=-1)[0]
    alist.append([])
    for i in range(len(x)):
        alist[-1].append(-x[i] * 2 * db)
    return c



Now we implement the full QITE protocol. It will be good to have a big picture of what should be done. We step through imaginary time and at each time step, we should first measure the expectation values of the pauli matrices $\sigma$. This is indicated in the first two lines of the for loop. Using the relevant expectation values, we obtain the coefficients $a[m]$ in equation (2.5) of Lecture 12 using the update rule in equation (2.9-2.11). We store it in a list and use this to propagate our state. Note we have to always reconstruct our state for each new measurements we make. Finally, we measure the current energy values. Let us construct the required functions starting with **get_expectation**. We make use of the **measure** function we constructed earlier. The key here is to propagate our state using the coefficients in alist. We do this using the **propagate** function.

In [9]:
import matplotlib.pyplot as plt
# from pyquil.gates import *
# from pyquil.noise import estimate_bitstring_probs
# from pyquil import Program,get_qc

def ansatz(p, qbits):
    None

def measure_energy(alist, shots, qc, qbits, hm_list):
    # Measure the energy at the end of each time step
    Energy = 0
    Nterms = len(hm_list)
    for i in range(len(hm_list)):
        hm = hm_list[i]
        for j in range(len(hm)):
            p = Program()
            p.wrap_in_numshots_loop(shots)
            # declare readout register with 1 bit
            ro = p.declare('ro', 'BIT', 1)
            ansatz(p, qbits)
            propagate(p, alist, qbits)
            pauli_pair = (hm[j][0][0], hm[j][0][1])
            idx = pauli_pair_dict[pauli_pair]
            tmp = hm[j][1][0] * measure(p, ro, idx, qc, qbits)
            print(tmp)
            Energy += tmp

    return Energy

def get_expectation(alist, shots, qc, qbits):
    # Obtain the expectation values of the Pauli string at each time step
    sigma_expectation = np.zeros([16], dtype=complex)
    for i in range(16):
        p = Program()
        p.wrap_in_numshots_loop(shots)
        ro = p.declare('ro','BIT',1)
        ansatz(p,qbits)
        propagate(p, alist, qbits)
        sigma_expectation[i] = measure(p, ro, i, qc, qbits)
    return sigma_expectation

def qite_step(alist, shots, qc, qbits, db, delta, hm_list):
    sigma_expectation = get_expectation(alist, shots, qc, qbits)
    for j in range(len(hm_list)):
        update_alist(sigma_expectation, alist, db, delta, hm_list[j]) # = norm
    return alist, sigma_expectation

def qite(qc, qbits, shots, db, delta, N, hm_list):
    E = np.zeros([N+1],dtype=complex)
    alist = []
    E[0] = measure_energy(alist, shots, qc, qbits, hm_list)
    print("Initial energy: ", E[0])
    # Qite main loop
    QITE_expectations = []
    for i in range(1,N+1):
        alist, sigma_expectation = qite_step(alist, shots, qc, qbits, db, delta, hm_list)
        print(sigma_expectation.real.tolist())
        QITE_expectations.append(sigma_expectation.real.tolist())
        E[i] = measure_energy(alist, shots, qc, qbits, hm_list)
#         print("a list: ", alist)
        print("energy: ", E[i])
    print(full_sigma_expectation_list)
    return E, QITE_expectations

### Running QITE with 2-site 1D Hubbard Hamiltonian

The 2-site 1D Hubbard Hamiltonian, for a half-filled lattice (2 fermions) is:

$$ H = -t \sum_{\sigma} \left( a_{1 \sigma}^{\dagger} a_{2 \sigma} + a_{2 \sigma}^{\dagger} a_{1 \sigma} \right) + U \sum_{i=1}^{2} n_{i \uparrow} n_{i \downarrow} $$

We can solve for the ground state energy of this Hamiltonian exactly, since the Hilbert space is small enough. We expect the ground state wavefunction to be symmetric with spin up and spin down components, and the most general guess is:

$$| \psi \rangle = \alpha \left( a_{1 \uparrow}^{\dagger} a_{1 \downarrow}^{\dagger} + a_{2 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} \right) |0 \rangle + \beta \left( a_{1 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} + a_{1 \uparrow}^{\dagger} a_{2 \downarrow}^{\dagger} \right) |0 \rangle$$

Using the time independent Schrodinger equation $H | \psi \rangle = E | \psi \rangle$ we get the coupled equations:

$$ (E-U) \alpha + 2 t \beta = 0$$
$$2t \alpha + E \beta = 0$$

There are two solutions; the one with the lower energy is the ground state energy:

$$E_{0} = \frac{1}{2} \left( U - \sqrt{U^{2} + 16 t^{2}} \right)$$

For our simulation, we run QITE with $t=1$ and $U=2$, so our goal is to match the exact ground state energy of $E_{0} = -1.23607$

We run QITE with a two qubit QVM using the 2-site 1D Hubbard Hamiltonian, with fermionic operators mapped to Pauli matrices by the Jordan-Wigner transformation.

$$H = -t \left( X \otimes I + I \otimes X \right) + \frac{U}{2} \left( I + Z \otimes Z \right)$$

In [10]:
# ---- input parameters for qite
N = 40
shots = 1000 #8192
db = 0.05
# use a 2-qubit qvm
qc = '2q-qvm'
qbits = [0, 1]
# qbits = [1, 0]
# kinetic energy contribution
t = 1.
# potential energy contribution
U = 2.
print("t = ", t, ", U = ", U)
# construct hm_list to represent H = -t(X \otimes I + I \otimes X) + U/2 (I + Z \otimes Z)
hm_list = []
# the first term (hopping kinetic term) of H
hm_list.append([])
hm_list[0].append([[1, 0], [-t]])
hm_list[0].append([[0, 1], [-t]])

hm_list[0].append([[0, 0], [U/2]])
hm_list[0].append([[3, 3], [U/2]])

# # the second term (onsite potential term) of H
# hm_list.append([])
# hm_list[1].append([[0, 0], [U/2]])
# hm_list[1].append([[3, 3], [U/2]])
print("hm list: ", hm_list)
delta = 0.01
print("running qite...")
E, QITE_expectations = qite(qc, qbits, shots, db, delta, N, hm_list)

t =  1.0 , U =  2.0
hm list:  [[[[1, 0], [-1.0]], [[0, 1], [-1.0]], [[0, 0], [1.0]], [[3, 3], [1.0]]]]
running qite...
-0.07600000000000001
-0.008000000000000007
1.0
1.0
Initial energy:  (1.916+0j)
[1.0, 0.028000000000000025, -0.03200000000000003, 1.0, 0.0020000000000000018, 0.0020000000000000018, -0.03200000000000003, -0.010000000000000009, -0.02200000000000002, 0.016000000000000014, -0.010000000000000009, -0.02400000000000002, 1.0, -0.03600000000000003, -0.008000000000000007, 1.0]
-0.030000000000000027
-0.04400000000000004
1.0
0.984
energy:  (1.91+0j)
[1.0, 0.07600000000000001, -0.062000000000000055, 0.99, 0.03200000000000003, 0.05800000000000005, -0.02200000000000002, 0.19999999999999996, -0.10400000000000004, 0.008000000000000007, 0.01200000000000001, -0.03600000000000003, 1.0, 0.13599999999999995, -0.12000000000000005, 0.984]
-0.18999999999999995
-0.07400000000000001
1.0
0.968
energy:  (1.704+0j)
[1.0, 0.12799999999999995, -0.12599999999999995, 0.97, 0.13199999999999995, 0.2099999

KeyboardInterrupt: 

In [ ]:
# some trouble with numpy sometimes because of imaginary part... so plot only real part of E
plt.figure(figsize=[8,4], dpi=150)
plt.plot(np.arange(0,N+1)*db,E.real,'g-o',label='QITE')
plt.axhline(y=-1.23607, color='r',linestyle='--',label="Exact")
plt.title("QITE on 2-site 1D Hubbard Model")
plt.xlabel("Imaginary time")
plt.ylabel("Energy")
plt.grid()
plt.legend(bbox_to_anchor=(1.0,1.0))
plt.show()

In [ ]:
def get_diff_between_lists(a,b):
    assert len(a) == len(b), print("go away")
    diff = [0]*len(a)
    for i in range(len(a)):
        diff[i] = a[i] - b[i]
        if diff[i] < 0.1:
            diff[i] = 0
    return diff

In [ ]:
classical_QITE_expectations = [[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0], [1.0000000000000002, 0.5875945241404422, -3.923726203155922e-15, 0.6459204807786503, 0.5875945241404403, 0.18614824179461656, 1.923208955806588e-14, 0.740361779529068, -2.69598713963591e-14, -7.172215547990212e-15, 0.48735962872487193, -3.2253584905223524e-14, 0.645920480778652, 0.7403617795290697, -3.2283289709909248e-15, 0.3264921294805118], [1.0000000000000002, 0.7915157699881595, -4.50114172275724e-15, 0.39684414982008476, 0.807703302314515, 0.7301186398241034, 8.064375058956748e-15, 0.5790759043206679, -2.0829204142415845e-14, -1.5933092339981455e-14, 0.4647768352807887, -1.543649790066658e-14, 0.3627642055338454, 0.5562746556647854, -3.676858096534116e-15, -0.19538137806672345], [1.0000000000000004, 0.867890844399881, -9.966852348246363e-16, 0.2113027586298094, 0.8762641064466875, 0.932511660115753, 6.297996200421643e-15, 0.31682394858543883, -1.4526865025196367e-14, -1.50457307445826e-14, 0.4495738275325809, -4.830649043206626e-15, 0.1733337513322032, 0.2928705310262461, 8.94520744852386e-17, -0.3826069364368522], [1.0, 0.8880188716055775, 2.0330351936456983e-15, 0.10826332909334507, 0.8913457947745356, 0.9844414254068399, 6.764152738764449e-15, 0.15834753250739575, -9.003353243038227e-15, -1.0971762400685826e-14, 0.44687977717276817, -1.5586394044098758e-15, 0.07616520788239584, 0.138398452002092, 3.578088545897458e-16, -0.43168106585901606], [1.0, 0.8928214358513773, 2.7962561491165784e-15, 0.05659598624710549, 0.8940863521108894, 0.9964386327086504, 6.133396127874399e-15, 0.07853174072739721, -6.021291744334e-15, -8.103920227416014e-15, 0.44684088669794153, -6.25832860630621e-16, 0.030705324359796726, 0.06250556444375899, 9.867087427572582e-17, -0.4435117240644378], [0.9999999999999996, 0.8939861808798093, 2.7649329737229557e-15, 0.030964316730694608, 0.8944634843343948, 0.9991576449313755, 5.239392917506069e-15, 0.0397346809241898, -4.2993334885783714e-15, -6.1835505976849725e-15, 0.44702339925929313, -2.3622165405028584e-16, 0.010254545676892518, 0.02692975426155747, 3.8031869907506855e-17, -0.44632932183286467], [0.9999999999999996, 0.8942892673660956, 2.463871733732257e-15, 0.017995753102450385, 0.8944691271509906, 0.9997816208955173, 4.343942764968707e-15, 0.020846800470907756, -3.1795939147178743e-15, -4.785698198506902e-15, 0.44712734108314905, -8.111891762539558e-17, 0.0014565128414519846, 0.010623718266539295, 2.2253953329321182e-17, -0.44700348676932855], [0.9999999999999997, 0.8943771785536929, 2.081326562324718e-15, 0.011191812370160092, 0.8944449364286997, 0.9999317730677142, 3.537364020821794e-15, 0.011506490811894476, -2.421044802394975e-15, -3.74724913887168e-15, 0.44717357459738855, -2.7245722347595406e-17, -0.0020124310801445666, 0.003345526195933368, 1.1685298068811762e-17, -0.4471655880672515], [0.9999999999999998, 0.8944062953239316, 1.7170644042909359e-15, 0.007436300632527759, 0.8944318201095054, 0.9999723198354711, 2.858247865174943e-15, 0.006761708740481931, -1.8771105594280365e-15, -2.9571455911121956e-15, 0.44719356023742374, -1.0902014819034393e-17, -0.003104650990796348, 0.0002469732139926961, 3.762758436787099e-18, -0.4472042689642268]]

In [ ]:
assert len(QITE_expectations) == len(classical_QITE_expectations), print("...")

diff_matrix = [0] * N
# should be N
for i in range(N):
    a = QITE_expectations[i]
    b = classical_QITE_expectations[i]
    diff_matrix[i] = get_diff_between_lists(a,b)

print(np.array(diff_matrix))